<a href="https://colab.research.google.com/github/GGsimidaRazer/TikTok-summer-school/blob/main/%E7%AC%AC%E5%9B%9B%E8%AE%B2%E8%AF%BE%E5%90%8E%E4%BD%9C%E4%B8%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dppy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.9 MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand, randn
from scipy.special import softmax
from scipy.linalg import norm
from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear

%config InlineBackend.figure_format='retina'

## 随机产生100个候选视频
def produce_random_items():
  embeddings=randn(100,512) #100条视频，512维embedding
  embeddings=embeddings*(embeddings>0) #RELU
  embeddings=embeddings/norm(embeddings,axis=1,keepdims=True)
  weights=rand(100,512)
  #embeddings=softmax(embeddings,axis=1)
  scores=np.exp(np.sum(embeddings*weights,axis=1)) #模拟rank阶段的logistic回归
  u=np.diag(scores)
  L=embeddings.dot(embeddings.T)
  L=u.dot(L)
  L=L.dot(u)
  return L




1. 随机产生100个候选视频，有对应的embedding和相关分s。产生8个DPP打散后的结果。

In [ ]:
DPP=FiniteDPP('likelihood',
                **{'L': produce_random_items()})

              #由于不清楚8个DPP打散后的结果指的是进行8次采样还是单个窗口大小为8的采样，因此都写上了
DPP.flush_samples()
for i in range(8):
  DPP.sample_exact()             
print(DPP.list_of_samples)     #此处为8组DPP采样结果

DPP.flush_samples()
DPP.sample_exact_k_dpp(size=8) 
print(DPP.list_of_samples)    #此处为窗口大小为8的采样结果


[[92, 1, 33, 48, 41, 42, 71, 24, 37, 94, 79, 43, 58, 53, 23, 55, 51, 76, 65, 26, 78, 29, 36, 32, 27, 84, 35, 93, 46, 5, 82, 63, 74, 87, 4, 28, 56, 19, 57, 50, 10, 98, 49, 66, 45, 0, 91, 90, 17, 68, 18, 8, 6, 2, 86, 30, 25, 70, 34, 61, 99, 52, 83, 54, 11, 38, 73, 59, 96, 85, 13, 60, 88, 40, 72, 15, 64, 9, 81, 16, 95, 14, 69, 20, 12, 7, 21, 75, 47, 67, 80, 44, 89, 39, 31, 22, 62, 97, 3, 77], [79, 76, 57, 29, 68, 30, 12, 67, 18, 15, 33, 8, 69, 38, 49, 82, 9, 3, 42, 88, 7, 44, 5, 21, 16, 0, 36, 86, 90, 77, 58, 55, 73, 75, 60, 94, 87, 51, 48, 95, 72, 80, 96, 11, 70, 92, 4, 53, 6, 17, 78, 40, 28, 56, 63, 83, 23, 91, 84, 19, 27, 24, 34, 89, 39, 46, 35, 62, 59, 74, 65, 1, 98, 64, 97, 52, 50, 20, 85, 31, 81, 66, 43, 41, 25, 71, 93, 99, 26, 61, 37, 45, 47, 13, 54, 22, 14, 2, 32, 10], [66, 94, 23, 87, 44, 16, 49, 85, 19, 36, 1, 65, 62, 57, 14, 78, 52, 55, 31, 45, 3, 47, 13, 41, 73, 68, 60, 86, 5, 28, 42, 81, 34, 75, 38, 69, 48, 92, 59, 46, 53, 2, 80, 25, 40, 64, 71, 11, 89, 56, 90, 35, 72, 18, 12

2. 实现DPP greedy search算法

In [ ]:
class DPP_GreedySearch:
  def __init__(self,L):
    self.L=L
    self.c=np.zeros_like(L)
    self.d_square=np.copy(np.diagonal(L))
    self.j=np.argmax(self.d_square)
    self.item_count=L.shape[0]
    self.Yg=[self.j]
  def run(self,k=None):
    if not k:
      k = self.item_count
    Z=set(list(np.arange(0,self.item_count)))
    for count in range(k-1):  
      Z_d=Z.difference(set(self.Yg))
      for i in Z_d:                  #对ci和di进行更新
        e=(self.L[self.j,i]-np.dot(self.c[:count,i],self.c[:count,self.j].T)) / np.sqrt(self.d_square[self.j])
        self.c[count,i]=e
        self.d_square[i]=self.d_square[i] - e*e
      self.d_square[self.j] = 0

      self.j=np.argmax(self.d_square)           #选中下一个结果
      self.Yg.append(self.j)
      
    return self.Yg
dpp=DPP_GreedySearch(produce_random_items())
result=dpp.run(20)
print(result)





[8, 19, 18, 12, 15, 66, 27, 37, 81, 11, 43, 28, 94, 58, 52, 82, 65, 13, 32, 33]


3. 实现DPP beam search 算法

In [ ]:
import copy

class DPP_BeamSearch:
  def __init__(self,L,beam_size):
    self.bsize=beam_size
    self.L=L
    self.c=np.copy(np.tile(np.zeros_like(L),(beam_size,1,1)))
    
    self.Yg=[[] for _ in range(beam_size)]
    
    self.j=np.zeros(beam_size,dtype=int) #beam个j
    self.d_square=np.copy(np.tile(np.copy(np.diagonal(L)),(beam_size,1)))
    
    d_square=np.copy(np.diagonal(L))
    
    for i in range(beam_size):
      j=np.argmax(d_square)
      self.Yg[i].append(j)
      self.j[i]=j
      d_square[j]=0

    
    self.item_count=L.shape[0]
  def run(self,k=None):
    if not k:
      k = self.item_count
    Z=set(list(np.arange(0,self.item_count)))
    
    for count in range(k-1):
      
      j_score=np.zeros((self.bsize,self.bsize))
      j_pos=np.zeros((self.bsize,self.bsize),dtype=int)
      for b in range(self.bsize):   #分别对beam_size个概率图进行更新
        Z_d=Z.difference(set(self.Yg[b]))
        
        for i in Z_d:
          
          e=(self.L[self.j[b],i]-np.dot(self.c[b,:count,i],self.c[b,:count,self.j[b]].T)) / np.sqrt(self.d_square[b,self.j[b]])
          self.c[b,count,i]=e
          self.d_square[b,i]=self.d_square[b,i] - e*e
        self.d_square[b,self.j[b]]=0
      
        d_square=np.copy(self.d_square[b])
        
        for b1 in range(self.bsize):  #每个更新后的概率图再取beam_size个结果作为备选
          j_score[b,b1]=np.max(d_square)
          
          j_pos[b,b1]=np.argmax(d_square)
          d_square[j_pos[b,b1]]=0
      

      pos=np.zeros((self.bsize,2),dtype=int)
      d_square_copy=np.zeros_like(self.d_square)
      Yg_copy=[[] for _ in range(self.bsize)]
      c_copy=np.zeros_like(self.c)
      for b in range(self.bsize):   #确定最终的beam_size个结果
        pos[b]=np.argwhere(j_score==np.max(j_score))[0]
        
        j_score[pos[b,0],pos[b,1]]=0
        d_square_copy[b]=np.copy(self.d_square[pos[b,0]])
        c_copy[b]=np.copy(self.c[pos[b,0]])
        
        Yg_copy[b]=copy.deepcopy(self.Yg[pos[b,0]])
        self.j[b]=j_pos[pos[b,0],pos[b,1]] 
        Yg_copy[b].append(self.j[b])
      self.d_square=np.copy(d_square_copy) #依次将di ci yg更新为选中的结果
      self.c=np.copy(c_copy)

      self.Yg=copy.deepcopy(Yg_copy)
      
      
    return self.Yg[0]           #由于最先取出的是条件概率最大的，因此返回第一个结果
test=DPP_BeamSearch(produce_random_items(),5)
result=test.run()
print(result)



[15, 3, 50, 59, 96, 19, 2, 38, 28, 58, 80, 40, 79, 73, 61, 37, 27, 44, 4, 69, 33, 93, 46, 74, 63, 32, 34, 81, 85, 64, 0, 66, 92, 47, 41, 17, 65, 5, 56, 22, 7, 90, 51, 30, 54, 78, 76, 12, 29, 83, 84, 72, 68, 89, 95, 18, 75, 87, 8, 97, 71, 14, 86, 49, 77, 10, 36, 9, 53, 24, 23, 42, 52, 55, 67, 39, 62, 70, 82, 16, 35, 26, 21, 57, 6, 31, 91, 94, 45, 98, 25, 48, 20, 11, 88, 1, 60, 13, 99, 43]


4. 重复1w次模拟实验比较不同beam size下beam search算法和greedy算法的异同（统计计算复杂度，准确度）

In [69]:
from google.colab import files
import time
import pandas as pd

class Comparison:
  def __init__(self,beam_range):
    self.L=produce_random_items()
    self.brange=beam_range
    self.dpp_g=DPP_GreedySearch(self.L)
    for i in range(beam_range):
      setattr(self,'dpp_b_%d'%(i+1),DPP_BeamSearch(self.L,(i+1)))
  def run(self,k=None):
    stat={}
    start_time=time.time()
    r=self.dpp_g.run(k)
    end_time=time.time()
    stat['GreedySearch_time']=end_time-start_time
    stat['GreedySearch_result']=r
    for i in range(self.brange):
      start_time=time.time()
      r=getattr(self,'dpp_b_%d'%(i+1)).run(k)
      end_time=time.time()
      stat['BeamSearch_size{}_result'.format(i+1)]=r
      stat['BeamSearch_size{}_time'.format(i+1)]=end_time-start_time
    return stat
beamrange=10

names=['BeamSearch_size{}'.format(i+1) for i in range(beamrange)].append('GreedySearch')
names=pd.Series(names,dtype=str)
all_times=pd.DataFrame(index=names)
all_results=pd.DataFrame(index=names)
repeat=100
for j in range(repeat):
  times=pd.Series(dtype=float,index=names)
  results=pd.Series(index=names)
  com=Comparison(10)
  result=com.run(20)
  times['GreedySearch']=result['GreedySearch_time']
  results['GreedySearch']=result['GreedySearch_result']
  for i in range(beamrange):
    times['BeamSearch_size{}'.format(i+1)]=result['BeamSearch_size{}_time'.format(i+1)]
    results['BeamSearch_size{}'.format(i+1)]=result['BeamSearch_size{}_result'.format(i+1)]
  all_times.insert(loc=0,column=(repeat-j),value=times)
  all_results.insert(loc=0,column=(repeat-j),value=results)
all_times.to_csv('times.csv')
all_results.to_csv('results.csv')
files.download('times.csv')
files.download('results.csv')






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


                        1         2         3         4         5         6    \
GreedySearch       0.015280  0.015466  0.015030  0.015101  0.015066  0.015475   
BeamSearch_size1   0.020945  0.021972  0.021343  0.021307  0.021504  0.021122   
BeamSearch_size2   0.045074  0.042477  0.042403  0.042261  0.042253  0.042033   
BeamSearch_size3   0.052833  0.053650  0.053661  0.054863  0.055562  0.056488   
BeamSearch_size4   0.052202  0.051591  0.052300  0.063346  0.054531  0.051438   
BeamSearch_size5   0.065033  0.077644  0.123310  0.065747  0.064680  0.066186   
BeamSearch_size6   0.091436  0.080211  0.384954  0.081158  0.079531  0.080480   
BeamSearch_size7   0.108909  0.097494  0.168899  0.095893  0.094903  0.092584   
BeamSearch_size8   0.108810  0.110523  0.125091  0.466672  0.112333  0.108878   
BeamSearch_size9   0.125771  0.131629  0.123783  0.199090  0.127815  0.125452   
BeamSearch_size10  0.145570  0.143834  0.139798  0.148099  0.599842  0.145920   

                        7  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>